## Jak działa przeciążanie operatorów:

•  przeciążanie operatorów pozwala klasom przechwytywać operacje Pythona,

•  klasy mogą przeciążać wszystkie operacje wyrażeń w Pythonie,

•  klasy mogą również przeciążać wbudowane operacje, jak wyświetlanie znaków, 
wywołania funkcji, dostęp do atrybutów itp.,

•  przeciążanie operatorów pozwala klasom definiowanym przez  użytkownika działać w sposób zbliżony do typów wbudowanych,

•  przeciążanie jest implementowane przez definiowanie metod klas o specjalnych nazwach.

In [1]:
class Number:
    def __init__(self, start):                  # Wywoływana przy Number(start)
        self.data = start
    def __sub__(self, other):                   # Wywoływana przy instancja – inna
        return Number(self.data - other)        # Wynik jest nową instancją

In [2]:
X = Number(5)

In [3]:
Y = X - 2

In [4]:
Y.data

3

## Indeksowanie i wycinanie —  __ getitem__ i  __ setitem__

Po zdefiniowaniu w klasie metoda __ getitem__ będzie automatycznie użyta w przypadkach prób wydobycia elementów po indeksach. Na przykład jeśli klasa X wystąpi w kontekście indeksowania X[i], Python wywoła jej metodę __ getitem__, przekazując X w pierwszym argumencie, a indeks w drugim.

In [5]:
class Indexer:
    def __getitem__(self, index):
        return index**2

In [6]:
X = Indexer()

In [7]:
X[2]

4

In [8]:
for i in range(5):
    print(X[i], end = ', ')

0, 1, 4, 9, 16, 

In [9]:
L = [5, 6, 7, 8, 9]

In [10]:
L[slice(2,4)]  # pokazanie jak dziala funkcja slice

[7, 8]

In [11]:
L[slice(1, None)]

[6, 7, 8, 9]

In [12]:
L[slice(None, None, 2)]

[5, 7, 9]

In [13]:
class Indexer2:
    data = [5, 6, 7, 8, 9]
    def __getitem__(self, index):
        print('getitem: ', index)
        return self.data[index]

In [14]:
Z = Indexer2()

In [15]:
Z[0]

getitem:  0


5

In [16]:
Z[1]

getitem:  1


6

In [17]:
Z[1:]

getitem:  slice(1, None, None)


[6, 7, 8, 9]

In [18]:
Z[:-1]

getitem:  slice(None, -1, None)


[5, 6, 7, 8]

In [19]:
Z[::2]

getitem:  slice(None, None, 2)


[5, 7, 9]

## Iteracja po indeksie —  __ getitem__

In [20]:
class stepper:
    def __getitem__(self, i):
        return self.data[i]

In [21]:
X = stepper()

In [22]:
X.data = 'mielonka'

In [23]:
X[1]

'i'

In [24]:
for item in X:
    print(item, end = ' ')

m i e l o n k a 

In [25]:
'p' in X

False

In [26]:
[x for x in X]  # lista składana

['m', 'i', 'e', 'l', 'o', 'n', 'k', 'a']

In [27]:
list(map(str.upper, X))

['M', 'I', 'E', 'L', 'O', 'N', 'K', 'A']

In [28]:
(a, b, c, d, e, f, g, h) = X   # przypisanie do sekwencji

In [29]:
a, b, c

('m', 'i', 'e')

In [30]:
list(X), tuple(X), ''.join(X)

(['m', 'i', 'e', 'l', 'o', 'n', 'k', 'a'],
 ('m', 'i', 'e', 'l', 'o', 'n', 'k', 'a'),
 'mielonka')

In [31]:
X

## Iteratory zdefiniowane przez użytkownika: Po zdefiniowaniu mechanizmu __ iter__ i __ next__ klasy stają się iteratorami zdefiniowanymi przez użytkownika


In [32]:
class Squares:
    def __init__(self, start, stop):          # Zapisanie stanu przy utworzeniu
        self.value = start - 1
        self.stop = stop
    def __iter__(self):                       # Otrzymanie obiektu iteratora na wywolanie iter()
        return self
    def __next__(self):                           # Zwrócenie kwadratu z każdą iteracją
        if self.value == self.stop:            # Również wywoływane przez funkcję wbudowaną next
            raise StopIteration
        self.value += 1
        return self.value ** 2

In [33]:
for i in Squares(1,5):
    print(i, end = ' ')

1 4 9 16 25 

In [34]:
X = Squares(2,4)

In [35]:
I = iter(X)   # wywolanie __iter__

In [36]:
next(I)      # wywolanie __next__

4

In [37]:
next(I)      # wywolanie __next__

9

In [38]:
# X[1]   # nie zadzialo bo nie ma metody __getitem__

In [39]:
X = Squares(2,6)

In [40]:
[n for n in X]

[4, 9, 16, 25, 36]

In [41]:
[n for n in Squares(2,6)]

[4, 9, 16, 25, 36]

In [42]:
list(Squares(2,6))

[4, 9, 16, 25, 36]

In [43]:
def kwadraty(start, stop):
    for i in range(start, stop + 1):
        yield i ** 2

In [44]:
for i in kwadraty(2,6):    # praktycznie to samo, przy uzyciu generatora 'kwadraty'
    print(i, end = ' ')

4 9 16 25 36 

## Wiele iteracji po jednym obiekcie

In [45]:
S = 'ace'

In [46]:
for x in S:
    for y in S:
        print(x + y)

aa
ac
ae
ca
cc
ce
ea
ec
ee


In [47]:
class SkipIterator:                             # zwraca co drugi element
    def __init__(self, wrapped):
        self.wrapped = wrapped                  # Informacje o stanie iteratora
        self.offset = 0
    def __next__(self):
        if self.offset >= len(self.wrapped):    # Zakończenie iteracji
            raise StopIteration
        else:
            item = self.wrapped[self.offset]    # Inaczej zwrócenie elementu i pominięcie
            self.offset += 2 
            return item

In [48]:
class SkipObject:
    def __init__(self, wrapped):           # Zapisanie elementu, który ma być użyty
        self.wrapped = wrapped
    def __iter__(self):
        return SkipIterator(self.wrapped)  # Za każdym razem nowy iterator
if __name__ == '__main__':
    alpha = 'abcdef'
    skipper = SkipObject(alpha)            # Utworzenie obiektu pojemnika
    I = iter(skipper)                      # Utworzenie na nim iteratora
    print(next(I), next(I), next(I))     # Odwiedzenie wartości przesunięcia 0, 2, 4
    
    for x in skipper:                      # for automatycznie wywołuje __iter__
        for y in skipper:                  # Zagnieżdżone for za każdym razem wywołują __iter__
            print(x+y)                     # Każdy iterator ma własny stan i przesunięcie

a c e
aa
ac
ae
ca
cc
ce
ea
ec
ee


### Test przynależności — __ contains__, __ iter __i __ getitem __ :
Klasy mogą implementować metodę __contains__: gdy ta metoda jest dostępna, jest preferowana
i ma przewagę nad __iter__, która z kolei ma pierwszeństwo przed __getitem__. Metoda
__contains__ powinna definiować przynależność na zasadzie kluczy słownika (wykorzystując
szybkie wyszukiwanie) oraz jako mechanizm wyszukiwania w sekwencjach.

In [49]:
class Iters:
    def __init__(self, value):
        self.data = value
    def __getitem__(self, i):               # Metoda zastępcza do użycia przez iterację
        print(f'get[{i}]', end='')          # oraz do indeksowania i wycinania
        return self.data[i]
    def __iter__(self):                     # Metoda preferowana w iteracji
        print('iter => ', end='')            # Pozwala na użycie tylko jednego iteratora
        self.ix = 0
        return self
    def __next__(self):
        print('next:', end='')
        if self.ix == len(self.data): raise StopIteration
        item = self.data[self.ix]
        self.ix += 1
        return item
    def __contains__(self, x):              # Metoda preferowana w operacji 'in'
        print('contains: ', end='')
        return x in self.data

In [50]:
X = Iters([1, 2, 3, 4, 5])                  # Utworzenie instancji
print(3 in X)                               # 'in' przywoluje metode contains z klasy
for i in X:                                 # Pętle for
    print(i, end=' | ')

contains: True
iter => next:1 | next:2 | next:3 | next:4 | next:5 | next:

In [51]:
print()
print([i ** 2 for i in X])                  # Inne konteksty iteracyjne
print(list(map(bin, X)))


iter => next:next:next:next:next:next:[1, 4, 9, 16, 25]
iter => next:next:next:next:next:next:['0b1', '0b10', '0b11', '0b100', '0b101']


In [52]:
I = iter(X)                                 # Ręczna iteracja (demonstracja mechanizmu stosowanego
                                            # w kontekstach iteracyjnych)
while True:
    try:
        print(next(I), end=' | ')
    except StopIteration:
        break

iter => next:1 | next:2 | next:3 | next:4 | next:5 | next:

### W powyzszym przykladzie metoda __ contains__ przechwytuje operację testu 'in', metoda __ iter__ obsługuje konteksty iteracyjne, w których wywoływana jest metoda __ next__, natomiast metoda __ getitem__ nie jest nigdy wywoływana.

### w przypadku, gdy zostanie zakomentowana metoda __contains__ — test przynależności jest realizowany przez metodę __iter__.

### po zakomentowaniu metod __contains__ i __iter__ —  wykorzystywana będzie metoda __getitem__, wywoływana z kolejnymi indeksami w kontekście testu przynależności oraz w iteracjach.

In [53]:
X = Iters ('123')
X[0]

get[0]

'1'

In [54]:
X[1:]

get[slice(1, None, None)]

'23'

### Metody __ getattr __ oraz __ setattr __ przechwytują referencje do atrybutów

Metoda __getattr__ przechwytuje atrybuty wywołane za pomocą składni kwalifikującej.
Jest wywoływana z nazwą atrybutu jako łańcuchem znaków zawsze, gdy próbujemy
zapisać  w  składni  kwalifikującej  instancję  z  niezdefiniowaną  (nieistniejącą)  nazwą  atrybutu.
Nie jest wywoływana, kiedy Python może odnaleźć atrybut po drzewie dziedziczenia. Ze względu na to zachowanie __getattr__ przydaje się jako punkt zaczepienia dla odpowiadania na żądania atrybutów w sposób ogólny. Przykład poniżej:

In [55]:
class Empty:
    def __getattr__(self, atr):
        if atr == 'age':
            return 18
        else:
            raise AttributeError(f'{atr}')

In [56]:
X = Empty()

In [57]:
X.age

18

In [58]:
# X.name  # AttributeError

### Metoda __ setattr __ przechwytuje wszystkie przypisania  atrybutów.  Jeśli  jest  ona  zdefiniowana,  *self.atrybut  =  wartość*  staje  się *self.__ setattr __('atrybut',  wartość)*.   Przypisanie do dowolnych atrybutów self wewnątrz wywołania metody __ setattr __ ponownie wywołuje __ setattr __, powodując nieskończoną pętlę rekurencji (i w końcu wyjątek przepełnienia stosu).  Aby  korzystać  z  tej  metody,  musimy  pamiętać,  że  przypisuje  ona dowolne atrybuty instancji, indeksując omówiony w kolejnym podrozdziale słownik atrybutów. Należy używać *self.__ dict __['name'] = x* zamiast *self.name = x*.

In [59]:
class accesscontrol:
    def __setattr__(self, attr, value):
        if attr == 'age':
            self.__dict__[attr] = value
        else:
            raise AttributeError(f'{attr} nie jest dozwolony')

In [60]:
X = accesscontrol()

In [61]:
X.age = 18

In [62]:
X.age

18

In [63]:
#X.name   #  nie wiem czemu nie zwraca wyjatku z metody setattr

### Emulowanie prywatności w atrybutach instancji

In [64]:
class PrivateExc(Exception): pass

In [65]:
class Privacy:
    def __setattr__(self, attrname, value):  # dla self.attrname = value
        if attrname in self.privates:
            raise PrivateExc(attrname, self)
        else:
            self.__dict__[attrname] = value   # pętla self.attrname = value

In [66]:
class Test1(Privacy):
    privates = ['age']

In [67]:
class Test2(Privacy):
    privates = ['name', 'pay']
    def __init__(self):
        self.__dict__['name'] = 'Amadeusz'

In [68]:
t1 = Test1()

In [69]:
t2 = Test2()

In [70]:
t1.name = "Edek"

In [71]:
# t2.name = 'Gienek'   # nie pozwala nadpisac imienia Amadeusz

In [72]:
#t1.age = 30  # nie pozwala ustawic wlasnego atrybutu age

In [73]:
t2.age = 40

### Metody  __ repr __ oraz  __ str __  zwracają reprezentacje łańcuchów znaków

In [74]:
class adder:
    def __init__(self, value=0):
        self.data = value                  # Inicjalizacja zmiennej data
    def __add__(self, other):
        self.data += other                 # Dodanie zmiennej other w miejscu

In [75]:
x = adder()

In [76]:
print(x)  # wyskakuje obiekt

In [77]:
class rpr(adder):
    def __repr__(self):
        return f'klasa rpr({self.data})'

In [78]:
x = rpr(2)

In [79]:
x + 1

In [80]:
x

klasa rpr(3)

In [81]:
str(x), repr(x)

('klasa rpr(3)', 'klasa rpr(3)')

In [82]:
print(x)

klasa rpr(3)


###  • __ str __ próbują w pierwszym rzędzie wykorzystać operacje wyświetlania przyjazne dla użytkownika, takie jak instrukcja print czy wbudowana funkcja str. Metoda __str__ powinna zwracać ciąg znaków przyjazny dla użytkownika.
### • __ repr __ wykorzystywana jest we wszystkich innych przypadkach: do zwracania wartości w sesji interaktywnej, wyniku wywołania funkcji repr, jak również do wyświetlania w sytuacji, gdy obiekt nie implementuje metody __ str __. Metoda __ repr __ powinna z reguły zwracać łańcuch znaków przypominający kod źródłowy, który można wykorzystać do odtworzenia obiektu oraz jako informację dla programistów, zawierającą dodatkowe informacje o obiekcie.

In [83]:
class st(adder):
    def __str__(self):
        return f'Wartość: {self.data}'

In [84]:
z = st(4)

In [85]:
z  # domyslnie wykonałby metode __repr__, ale jej nie ma w klasie

In [86]:
print(z)  # wykonuje metode __str__

Wartość: 4


In [87]:
class rprstr(adder):
    def __str__(self):
        return f'Wartosc z metody __str__: {self.data}'
    def __repr__(self):
        return f'Wartosc z metody __repr__: {self.data}'

In [88]:
a = rprstr(5)

In [89]:
a+1

In [90]:
a

Wartosc z metody __repr__: 6

In [91]:
print(a)

Wartosc z metody __str__: 6


### uwaga:  metoda __str__ jest używana wyłącznie w przypadku, gdy obiekt jest dostępny bezpośrednio. Jeśli jest zagnieżdżony w innym obiekcie, do jego wyświetlenia zostanie użyta metoda __repr__.

In [92]:
class Prstr:
    def __init__(self, val):
        self.val = val
    def __str__(self):                  # Używany dla instancji
        return str(self.val)            # Przekształcenie na ciąg znaków

In [93]:
class Prrepr:
    def __init__(self, val):
        self.val = val
    def __repr__(self):                  # Używany dla instancji
        return str(self.val)            # Przekształcenie na ciąg znaków

In [94]:
ps = [Prstr(2), Prstr(3)]

In [95]:
for x in ps: print(x)

2
3


In [96]:
print(ps)    # metoda string nie zadzialala po zagniezdzeniu klasy w instancji

[<__main__.Prstr object at 0x0000000005062C70>, <__main__.Prstr object at 0x0000000005062730>]


In [97]:
pr = [Prrepr(2), Prrepr(3)]

In [98]:
for x in pr: print(x)

2
3


In [99]:
print(pr)  # metoda repr dziala

[2, 3]


## Metoda __ radd __ obsługuje dodawanie prawostronne i modyfikację w miejscu (jest wywolywany przez Pythona, gdy tylko po prawej stronie '+' jest instancja klasy, w pozostalych przypadkach wywolywana jest metoda __ add__)

In [100]:
class Commuter:
    def __init__(self, val):
        self.val = val
    def __add__(self, other):
        print('add', self.val, other)
        return other + self.val
    def __radd__(self, other):
        print('radd', self.val, other)
        return other + self.val

In [101]:
x = Commuter(88)

In [102]:
y = Commuter(99)

In [103]:
x+1

add 88 1


89

In [104]:
1 + x

radd 88 1


89

In [105]:
y + 2

add 99 2


101

In [106]:
2 + y

radd 99 2


101

In [107]:
x + y

add 88 <__main__.Commuter object at 0x0000000005099BB0>
add 99 88


187

#### Uwaga:  Kiedy w wyrażeniach mieszanych dodawane są instancje instancje różnych klas, Python wybiera metody z klasy instancji znajdującej się po lewej stronie.

### Dodawanie w miejscu:
#### Aby obiekt obsłużył operację dodawania w miejscu +=, należy zaimplementować metodę __ iadd __ lub __ add __. Ta druga jest stosowana w przypadku, gdy klasa nie obsługuje pierwszej. Klasa Commuter omawiana w poprzednim punkcie miała zaimplementowaną tę metodę właśnie z myślą o operacji +=, ale __ iadd __ oferuje wydajniejsze modyfikacje w miejscu

In [108]:
class Numiadd:
    def __init__(self, val):
        self.val = val
    def __iadd__(self, other):             # __iadd__ wywołuje: x += y
        self.val += other                  # Z reguły zwraca self
        return self

In [109]:
x = Numiadd(5)

In [110]:
x += 3

In [111]:
x += 2

In [112]:
x.val

10

In [113]:
class Numadd:
    def __init__(self, val):
        self.val = val
    def __add__(self, other):              # __add__: x = (x + y)
        return Numadd(self.val + other)    # Przeniesienie typu na wynik

In [114]:
z = Numadd(5)

In [115]:
z += 2

In [116]:
z += 3

In [117]:
z.val

10

### Metoda __ call __  przechwytuje wywołania

In [118]:
class Calle:
    def __call__(self, *pargs, **kargs):   # klasa przyjmuje argumenty jak funkcja
        print('Wywołanie: ', pargs, kargs)

In [119]:
C = Calle()

In [120]:
C(1,2,3)

Wywołanie:  (1, 2, 3) {}


In [121]:
C(1,2,3, x=4, y=5)

Wywołanie:  (1, 2, 3) {'x': 4, 'y': 5}


In [122]:
class Prod:
    def __init__(self, value):         # Przyjmuje jeden argument
        self.value = value
    def __call__(self, other):         
        return self.value * other

In [123]:
x = Prod(2)                             # "Zapamiętuje" wartość 2
x(3)                                    # 3 (przekazane) * 2 (zapamiętane)

6

In [124]:
x(4)

8

### porownania: <, >, <=, >=, ==, !=:

In [125]:
class C:
    data = 'spam'
    def __gt__(self, other):               
        return self.data > other
    def __lt__(self, other):
        return self.data < other

In [126]:
X = C()

In [127]:
print(X > 'ham')

True


In [128]:
print(X < 'ham')

False


###  Testy logiczne — __ bool __ i __ len __:
###  Python wywoła metodę __ bool __, aby uzyskać bezpośrednią interpretację instancji, a jeśli metoda ta nie jest zdefiniowana, wywoła metodę __ len __, aby określić reprezentację obiektu na podstawie tego, czy zawiera elementy (True), czy też jest pusty (False).

In [129]:
class Tr:
    def __bool__(self): return True

In [130]:
X = Tr()

In [131]:
if X: print("prawda")

prawda


In [132]:
class F:
    def __bool__(self): return False

In [133]:
X = F()

In [134]:
bool(X)

False

In [135]:
class Bol:
    def __len__(self): return 0

In [136]:
Z = Bol()

In [137]:
if not Z: print('niet')

niet


In [138]:
class Truth:
    def __bool__(self): return True          # python 3.0 najpierw wywołuje __bool__
    def __len__(self): return 0      

In [139]:
Y = Truth()

In [140]:
bool(Y)

True

In [141]:
class Prawda: pass  # gdy nie jest zdefiniowana __len__ ani __bool_, obiekt zostanie zinterpretowany jako wartość True 

In [142]:
P = Prawda()

In [143]:
bool(P)

True

### Destrukcja obiektu — __del__

In [144]:
class Life:
    def __init__(self, name = 'nieznajomy'):
        print('witaj', name)
        self.name = name
    def __del__(self):
        print('żegnaj', self.name)

In [145]:
L = Life()

witaj nieznajomy


In [146]:
L = 'LOL'  # zmienna stracila referencje do klasy, wiec obiekt jest usuwany z pamieci

żegnaj nieznajomy
